### ZONAL STATISTICS AND MERGE ALL DATA FROM NOTEBOOKS 1 THRU 4

In [6]:
# Modules.
import pandas as pd
import numpy as np
from pathlib import Path

In [7]:
# Paths.
acs_path      = Path("data/acs/acs_socioeconomic_tract_2022.csv")
nlcd_path     = Path("data/raster/nlcd_calc_tracts.csv")
addfeat_path  = Path("data/additional_features/nyc_tracts_new_variables.csv")
heat_path     = Path("data/heat/JFK_2025_JJA_extreme_heat_90.csv")
calls_path    = Path("data/nyc_311/nyc_311_tract_day_2025.csv")

out_daily     = Path("data/model/nyc_tract_summer_2025_daily.csv")
out_weekly    = Path("data/model/nyc_tract_summer_2025_weekly.csv")
out_tract     = Path("data/model/nyc_tract_obs_summer_2025_cross_section.csv")

In [8]:
# Load data.
acs = pd.read_csv(acs_path, dtype = {"GEOID": str})
nlcd = pd.read_csv(nlcd_path, dtype = {"GEOID": str})
addfeat = pd.read_csv(addfeat_path, dtype = {"GEOID": str})
heat = pd.read_csv(heat_path)
calls = pd.read_csv(calls_path, dtype = {"GEOID": str})

In [9]:
# Align data dates.
heat["DATE"] = pd.to_datetime(heat["DATE"])
heat["EXTREME_HEAT"] = heat["EXTREME_HEAT"].str.strip().str.lower()

# Convert to binary.
heat["EXTREME_HEAT"] = heat["EXTREME_HEAT"].map({"yes": 1, "no": 0}).astype("int")
calls["DATE"] = pd.to_datetime(calls["DATE"])

In [10]:
# Merge panels.
# Left join because calls define the tract × date panel.
daily = calls.merge(heat, on = "DATE", how = "left")

# Merge ACS, NLCD, additional features (tract-level).
daily = daily.merge(acs, on = "GEOID", how = "left")
daily = daily.merge(nlcd, on = "GEOID", how = "left")
daily = daily.merge(addfeat, on = "GEOID", how = "left")

# Dependent variable.
daily["QOL_RATE_1K"] = (daily["QOL_CALLS"] / daily["TOTAL_CALLS"]) * 1000
daily["QOL_RATE_10K"] = (daily["QOL_CALLS"] / daily["TOTAL_CALLS"]) * 10000
daily["LOG_QOL_RATE_1K"] = np.log(daily["QOL_RATE_1K"] + 1)
daily["LOG_QOL_RATE_10K"] = np.log(daily["QOL_RATE_10K"] + 1)

# Fill missing heat days with 0.
daily["EXTREME_HEAT"] = daily["EXTREME_HEAT"].fillna(0).astype(int)

# Week calculation.
daily["WEEK"] = daily["DATE"].dt.isocalendar().week.astype(int)

In [11]:
# Weekly panel.
weekly = daily.groupby(["GEOID", "WEEK"], as_index=False).agg(
    TOTAL_CALLS_WEEK=("TOTAL_CALLS", "sum"),
    QOL_CALLS_WEEK=("QOL_CALLS", "sum"),
    EXTREME_DAYS_WEEK=("EXTREME_HEAT", "sum"),
    MEDIAN_TMAX_F=("TMAX_F", "median")
)

# Week-level binary heat flags
weekly["EXTREME_WEEK_1"] = (weekly["EXTREME_DAYS_WEEK"] >= 1).astype(int)
weekly["EXTREME_WEEK_2"] = (weekly["EXTREME_DAYS_WEEK"] >= 2).astype(int)

# Merge tract-level predictors
weekly = weekly.merge(acs, on="GEOID", how="left")
weekly = weekly.merge(nlcd, on="GEOID", how="left")
weekly = weekly.merge(addfeat, on="GEOID", how="left")

# Dependent variable.
weekly["QOL_RATE_1K"] = (
    (weekly["QOL_CALLS_WEEK"] / weekly["TOTAL_CALLS_WEEK"]) * 1000
)

weekly["QOL_RATE_10K"] = (
    (weekly["QOL_CALLS_WEEK"] / weekly["TOTAL_CALLS_WEEK"]) * 10000
)

weekly["LOG_QOL_RATE_1K"] = np.log(
    ((weekly["QOL_CALLS_WEEK"] / weekly["TOTAL_CALLS_WEEK"]) * 1000) + 1
)

weekly["LOG_QOL_RATE_10K"] = np.log(
    ((weekly["QOL_CALLS_WEEK"] / weekly["TOTAL_CALLS_WEEK"]) * 10000) + 1
)

In [12]:
# Weekly heat flags to daily panel.
daily = daily.merge(
    weekly[["GEOID", "WEEK", "EXTREME_WEEK_1", "EXTREME_WEEK_2"]],
    on=["GEOID", "WEEK"],
    how="left"
)

# If a tract/day had no heat days recorded → set weekly flags to 0
daily["EXTREME_WEEK_1"] = daily["EXTREME_WEEK_1"].fillna(0).astype(int)
daily["EXTREME_WEEK_2"] = daily["EXTREME_WEEK_2"].fillna(0).astype(int)

In [22]:
out_daily.parent.mkdir(parents=True, exist_ok=True)

daily.to_csv(out_daily, index=False)
print("Saved DAILY panel →", out_daily)

weekly.to_csv(out_weekly, index=False)
print("Saved WEEKLY panel →", out_weekly)

Saved DAILY panel → data\model\nyc_tract_summer_2025_daily.csv
Saved WEEKLY panel → data\model\nyc_tract_summer_2025_weekly.csv


In [13]:
# Collapse heat to tract-level summary.
heat_summary = (
    daily.groupby("GEOID", as_index=False)
    .agg(
        EXTREME_DAYS_SUMMER=("EXTREME_HEAT","sum"),
        TOTAL_DAYS_SUMMER=("EXTREME_HEAT","count"),
        MEAN_TMAX_F_SUMMER=("TMAX_F","mean")
    )
)

heat_summary["PCT_EXTREME_HEAT_DAYS"] = (
    heat_summary["EXTREME_DAYS_SUMMER"] / heat_summary["TOTAL_DAYS_SUMMER"]
)

heat_summary["EXTREME_HEAT_1"] = (heat_summary["EXTREME_DAYS_SUMMER"] >= 1).astype(int)
heat_summary["EXTREME_HEAT_2"] = (heat_summary["EXTREME_DAYS_SUMMER"] >= 2).astype(int)

# Collapse QOL to tract-level summaries.
calls_summary = (
    daily.groupby("GEOID", as_index=False)
         .agg(
             TOTAL_CALLS_SUMMER=("TOTAL_CALLS","sum"),
             QOL_CALLS_SUMMER=("QOL_CALLS","sum")
         )
)

calls_summary["QOL_RATE_1K"]  = (calls_summary["QOL_CALLS_SUMMER"] / calls_summary["TOTAL_CALLS_SUMMER"]) * 1000
calls_summary["QOL_RATE_10K"] = (calls_summary["QOL_CALLS_SUMMER"] / calls_summary["TOTAL_CALLS_SUMMER"]) * 10000

# Merge all tract-level datasets.
tract = (heat_summary
    .merge(calls_summary, on="GEOID", how="left")
    .merge(acs, on="GEOID", how="left")
    .merge(nlcd, on="GEOID", how="left")
    .merge(addfeat, on="GEOID", how="left")
)

In [14]:
# Save TRACT-ONLY panel
tract.to_csv(out_tract, index=False)
print("Saved CROSS-SECTION panel →", out_tract)

print("All panels completed.")

Saved CROSS-SECTION panel → data\model\nyc_tract_obs_summer_2025_cross_section.csv
All panels completed.
